In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle

def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )
def get_datasets(path_chile, path_italia, binary_target, columns_drop_NANs):
    df_train, _ = extract_transform(path_chile,                 OPBG, columns_drop_NANs,binary_target)
    df_train.drop('código', axis=1, inplace=True)
    df_test, df_test_raw  = extract_transform(path_italia, OPBG, columns_drop_NANs,binary_target)
    Rome_mask = df_test_raw['código'].str.startswith('R')

    df_test_FLOR  = encode_and_convert (df_test[~Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
    df_test_ROME  = encode_and_convert (df_test[Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
    df_test_FLOR[binary_target] = df_test_FLOR[binary_target].astype('int8').astype('category')
    df_test_ROME[binary_target] = df_test_ROME[binary_target].astype('int8').astype('category')


    compare_dataframes(df_test_ROME, df_test_FLOR) == compare_dataframes(df_train, df_test_ROME)

    return df_train, df_test_ROME, df_test_FLOR
OPBG = [ 'código','age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein','glicemia']

binary_target= 'Alpha-Fet'
df_chile, df_rome, df_flor = get_datasets(
    path_chile= "data/tirosinemia.csv", 
    path_italia=  "data/tirosinemia_italia.csv", 
    binary_target= binary_target, 
    columns_drop_NANs= ['alfa-fetoprotein', 'sca (urine)'])

23/06/08 09:17:32 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 100.64.221.100 instead (on interface tailscale0)
23/06/08 09:17:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/08 09:17:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/08 09:17:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.
Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


In [2]:
df_chile.to_parquet("data/df_chile.parquet.gzip")
df_rome.to_parquet("data/df_rome.parquet.gzip")
df_flor.to_parquet("data/df_flor.parquet.gzip")

In [3]:
%%time





def launch_to_ray(df_train,binary_target, save_to ,n_trials=150 ,Independent_testset = False, df_test = None):
    ray.shutdown()

    ray_df_train = ray.put(df_train)
    ray_df_test  = ray.put(df_test)
    features : list[str] = df_train.columns.tolist()
    features.remove(binary_target)
    #return ray_df_train, ray_df_test, features
    
    studies = make_multiple_studies(
        ray_df_train, 
        features = features,
        targets = [binary_target],
        n_trials=n_trials,
        Independent_testset = Independent_testset,
        Independent_testset_df = ray_df_test
    )

    S = ray.get(studies)
    with open(save_to + '.pickle', 'wb') as handle:
        pickle.dump(S, handle, protocol=pickle.HIGHEST_PROTOCOL)  
    
    ray.shutdown()



launch_to_ray(df_train = df_chile , binary_target= binary_target, save_to = "studies_chile", Independent_testset = False)
launch_to_ray(df_train = df_chile , binary_target= binary_target, save_to = "studies_rome", Independent_testset = True, df_test = df_rome)
launch_to_ray(df_train = df_chile , binary_target= binary_target,save_to= "studies_flor" ,Independent_testset = True, df_test = df_flor)



2023-06-08 09:17:36,672	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(make_a_study pid=3106597) [I 2023-06-08 09:17:38,147] A new study created in memory with name: sca (urine) (Alpha-Fet)
(make_a_study pid=3106599) [I 2023-06-08 09:17:38,172] A new study created in memory with name: ntbc levels (dbs) (Alpha-Fet)
(make_a_study pid=3106596) [I 2023-06-08 09:17:38,174] A new study created in memory with name: age at diagnosis (months) (Alpha-Fet)
(make_a_study pid=3106609) [I 2023-06-08 09:17:38,147] A new study created in memory with name: ntbc dosis mg/kg/day (Alpha-Fet)
(make_a_study pid=3106597) [I 2023-06-08 09:17:52,539] Trial 4 finished with values: [0.9090909090909091, 0.0] and parameters: {'seed': 3737, 'max_depth': 8, 'eta': 0.4373184305960064, 'subsample': 0.32941922810628227, 'lambda': 0.5577265724073561, 'alpha': 0.856992722880673, 'scale_pos_weight': 1.7115323478942064, 'kfold_splits': 2}. 
(make_a_study pid=3106597) [I 2023-06-

CPU times: user 58.9 s, sys: 1min, total: 1min 59s
Wall time: 37min 37s


In [4]:
# with open('chile.pickle', 'wb') as handle:
#     pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)    